<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [2]:
!pip3 install folium
!pip3 install wget
!pip3 install pandas

Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [3]:
import folium
import wget
import pandas as pd

In [4]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [7]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [8]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [10]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example, 


In [11]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle. 


Now, let's add a circle for each launch site in data frame `launch_sites`


_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [12]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


In [13]:
for _, row in launch_sites_df.iterrows():
    lat = float(row['Lat'])
    lon = float(row['Long'])
    name = str(row['Launch Site'])

    # Circle to highlight the site
    folium.Circle(
        location=[lat, lon],
        radius=1000,       # ~1 km highlight
        color=None,
        fill=True,
        fill_opacity=0.5,
        popup=name
    ).add_to(site_map)

    # Marker with the site name
    folium.Marker(
        location=[lat, lon],
        popup=name,
        icon=folium.Icon()
    ).add_to(site_map)

site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Are all launch sites in proximity to the Equator line?
- Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


- Are all launch sites in proximity to the Equator line?
Not all sites are near the Equator, but they are all in the southern U.S. region — closer to the Equator than most continental U.S. territory, chosen strategically for efficiency
- Are all launch sites in very close proximity to the coast?
Yes — all launch sites are coastal or within a few kilometers of the coastline, chosen deliberately for safety and trajectory optimization.

| Aspect                   | Observation                   | Reason                                                     |
| ------------------------ | ----------------------------- | ---------------------------------------------------------- |
| **Proximity to Equator** | Moderately close (≈ 26–35° N) | Gains partial Earth-rotation velocity advantage            |
| **Proximity to Coast**   | Very close (all coastal)      | Enables safe over-ocean flight paths and debris drop zones |


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [14]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [15]:
marker_cluster = MarkerCluster()


_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [19]:
spacex_df.head()

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS LC-40,28.562302,-80.577356,0
2,CCAFS LC-40,28.562302,-80.577356,0
3,CCAFS LC-40,28.562302,-80.577356,0
4,CCAFS LC-40,28.562302,-80.577356,0


In [ ]:
# Assign marker colors based on the class column
# Success (1) → green, Failure (0) → red

spacex_df['marker_color'] = spacex_df['class'].apply(
    lambda x: 'green' if x == 1 else 'red')

# Preview the updated DataFrame
#spacex_df.head()

In [21]:
spacex_df.head()

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


In [22]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [24]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# Ensure we have a marker_color column: success (1) -> green, failure (0) -> red
if 'marker_color' not in spacex_df.columns:
    spacex_df['marker_color'] = spacex_df['class'].map({1: 'green', 0: 'red'}).fillna('gray')

# For each row in spacex_df, create a Marker and add to the cluster
for _, row in spacex_df.iterrows():
    status = 'Success' if row['class'] == 1 else 'Failure'
    popup_text = f"{row['Launch Site']} — {status}"

    marker = folium.Marker(
        location=[float(row['Lat']), float(row['Long'])],
        popup=popup_text,
        # white base pin, icon colored by success/failure
        icon=folium.Icon(color='white', icon_color=row['marker_color'], icon='rocket', prefix='fa')
    )
    marker_cluster.add_child(marker)

site_map

In [23]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [30]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [29]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [ ]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [ ]:
from folium.plugins import MousePosition
import folium
import math

# 1) Add MousePosition so you can read latitude/longitude anywhere you hover
mp = MousePosition(
    position='topright',
    separator=' , ',
    prefix='Lat, Lon:',
    lat_formatter="function(num) {return L.Util.formatNum(num, 6);}",
    lon_formatter="function(num) {return L.Util.formatNum(num, 6);}"
)
site_map.add_child(mp)

# Optional: click to see a popup with lat/lon
site_map.add_child(folium.LatLngPopup())

# 2) After hovering/clicking the closest coastline on the map,
#    set the picked coordinates here:
#    (Replace with the values you read from MousePosition / click popup)
lat2 = lat2 if 'lat2' in globals() else 28.562302   # <-- example placeholder
lon2 = lon2 if 'lon2' in globals() else -80.577356   # <-- example placeholder

# 3) Define Haversine distance (in km) — no external libs needed
def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius (km)
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlmb = math.radians(lon2 - lon1)
    a = (math.sin(dphi/2)**2
         + math.cos(phi1) * math.cos(phi2) * math.sin(dlmb/2)**2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# 4) Find the nearest launch site to the selected coastline point
#    (If you already selected a specific site earlier, you can skip this and use that site’s coordinates)
dists = []
for _, r in launch_sites_df.iterrows():
    d = haversine_km(lat2, lon2, float(r['Lat']), float(r['Long']))
    dists.append((d, r['Launch Site'], float(r['Lat']), float(r['Long'])))
dists.sort(key=lambda x: x[0])

distance_km, site_name, site_lat, site_lon = dists[0]

# 5) Add a marker on the chosen coastline point with distance info
coast_popup = f"Closest coastline point<br>{lat2:.6f}, {lon2:.6f}<br><b>{distance_km:.2f} KM</b> to {site_name}"
folium.Marker(
    location=[lat2, lon2],
    popup=coast_popup,
    icon=folium.Icon(color='blue', icon='info-sign')
).add_to(site_map)

# 6) (Optional) Emphasize the launch site with a popup showing the distance as well
site_popup = f"{site_name}<br>{site_lat:.6f}, {site_lon:.6f}<br><b>{distance_km:.2f} KM</b> from coastline point"
folium.Marker(
    location=[site_lat, site_lon],
    popup=site_popup,
    icon=folium.Icon(color='green', icon='rocket', prefix='fa')
).add_to(site_map)

# Draw a line between coastline point and the launch site
folium.PolyLine(
    locations=[[lat2, lon2], [site_lat, site_lon]],
    weight=2
).add_to(site_map)

# (Optional) Add a small floating distance label near the midpoint
mid_lat = (lat2 + site_lat) / 2
mid_lon = (lon2 + site_lon) / 2
folium.map.Marker(
    [mid_lat, mid_lon],
    icon=folium.DivIcon(
        html=f'<div style="font-size:12px; background: rgba(255,255,255,0.8); padding:2px 6px; border-radius:4px;"><b>{distance_km:.2f} KM</b></div>'
    )
).add_to(site_map)

site_map

In [31]:
import math
import folium
from folium.plugins import MousePosition

# Show lat/lon under the mouse (helps you pick coastline coords)
mp = MousePosition(
    position='topright',
    separator=' , ',
    prefix='Lat, Lon:',
    lat_formatter="function(num) {return L.Util.formatNum(num, 6);}",
    lon_formatter="function(num) {return L.Util.formatNum(num, 6);}"
)
site_map.add_child(mp)
site_map.add_child(folium.LatLngPopup())

# --- 1) Set coastline coordinates (read them from MousePosition or click popup) ---
# Example from the comment: Lat: 28.56367  Lon: -80.57163
coastline_lat = 28.56367   # <-- replace with your picked value
coastline_lon = -80.57163  # <-- replace with your picked value

# --- 2) Get the current launch site coordinates ---
# If your notebook already defined launch_site_lat/launch_site_lon, reuse them.
# Otherwise, pick a site (here: the first row of launch_sites_df) or select by name.
try:
    launch_site_lat   # will raise NameError if not defined
    launch_site_lon
except NameError:
    # Option A: select a specific site by name (uncomment and adjust if you prefer)
    # site_row = launch_sites_df[launch_sites_df['Launch Site'].str.contains('LC-40', case=False)].iloc[0]

    # Option B: just take the first launch site in the dataframe
    site_row = launch_sites_df.iloc[0]
    launch_site_lat = float(site_row['Lat'])
    launch_site_lon = float(site_row['Long'])
    launch_site_name = str(site_row['Launch Site']) if 'Launch Site' in site_row else "Launch Site"
else:
    # If variables existed, try to infer a name for nicer popups
    launch_site_name = "Launch Site"

# --- 3) Distance function (Haversine, km) ---
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in km
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlmb = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlmb/2)**2
    return 2 * R * math.asin(math.sqrt(a))

# --- 4) Compute coastline distance ---
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(f"Distance from {launch_site_name} to coastline point: {distance_coastline:.2f} km")

# --- 5) Visualize on the map (optional but handy) ---
# Coastline point marker
folium.Marker(
    [coastline_lat, coastline_lon],
    popup=f"Coastline point<br>{coastline_lat:.6f}, {coastline_lon:.6f}<br><b>{distance_coastline:.2f} KM</b> from {launch_site_name}",
    icon=folium.Icon(color='blue', icon='info-sign')
).add_to(site_map)

# Launch site marker (only if you want to emphasize it)
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup=f"{launch_site_name}<br>{launch_site_lat:.6f}, {launch_site_lon:.6f}",
    icon=folium.Icon(color='green', icon='rocket', prefix='fa')
).add_to(site_map)

# Line between points
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    weight=2
).add_to(site_map)

# Midpoint distance label
mid_lat = (launch_site_lat + coastline_lat) / 2
mid_lon = (launch_site_lon + coastline_lon) / 2
folium.Marker(
    [mid_lat, mid_lon],
    icon=folium.DivIcon(
        html=f'<div style="font-size:12px;background: rgba(255,255,255,0.85); padding:2px 6px; border-radius:4px;"><b>{distance_coastline:.2f} KM</b></div>'
    )
).add_to(site_map)

site_map

Distance from CCAFS LC-40 to coastline point: 0.58 km


_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


In [ ]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

In [32]:
from folium.features import DivIcon

# --- Create a marker at the coastline point showing the distance to the launch site ---
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance_coastline)
    )
)

# Add the marker to your site_map
site_map.add_child(distance_marker)

# (Optional) Draw a line between coastline point and launch site for clarity
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    weight=2,
    color="#d35400"
).add_to(site_map)

# Display the updated map
site_map

_TODO:_ Draw a `PolyLine` between a launch site to the selected coastline point


In [ ]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

In [34]:
# Define the coordinates list connecting the launch site and the coastline
coordinates = [
    [launch_site_lat, launch_site_lon],
    [coastline_lat, coastline_lon]
]

# Create a folium.PolyLine object using the coordinates
lines = folium.PolyLine(
    locations=coordinates,
    weight=2,           # line thickness
    color='#d35400',    # optional color (orange)
    opacity=0.8
)

# Add the PolyLine to the map
site_map.add_child(lines)

# Display the map with the connecting line
site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


_TODO:_ Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [35]:
import math
import folium
from folium.features import DivIcon

# --- If you don't already have this ---
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # km
    from math import radians, sin, cos, asin, sqrt
    dphi = radians(lat2 - lat1)
    dlmb = radians(lon2 - lon1)
    phi1, phi2 = radians(lat1), radians(lat2)
    a = sin(dphi/2)**2 + cos(phi1) * cos(phi2) * sin(dlmb/2)**2
    return 2 * R * asin(sqrt(a))

# --- 1) Choose/define your launch site coordinates ---
# If you already defined launch_site_lat / launch_site_lon, this will be skipped.
try:
    launch_site_lat
    launch_site_lon
except NameError:
    # Example: pick a specific site by name (or use .iloc[0] to take the first)
    site_row = launch_sites_df[launch_sites_df['Launch Site'].str.contains('LC-39A', case=False)].iloc[0]
    launch_site_lat = float(site_row['Lat'])
    launch_site_lon = float(site_row['Long'])
    launch_site_name = str(site_row['Launch Site'])
else:
    launch_site_name = "Launch Site"

# --- 2) Set the closest POI (city/railway/highway) coordinates ---
# Replace with your measured/known coordinates
poi_name = "Closest City"     # e.g., "Titusville", "US-1", "Rail Yard"
poi_lat  = 28.612000          # <-- put your value
poi_lon  = -80.807000         # <-- put your value

# --- 3) Compute distance ---
distance_km = calculate_distance(launch_site_lat, launch_site_lon, poi_lat, poi_lon)

# --- 4) Add a marker at the POI with a popup ---
folium.Marker(
    [poi_lat, poi_lon],
    popup=f"{poi_name}<br>{poi_lat:.6f}, {poi_lon:.6f}<br><b>{distance_km:.2f} KM</b> from {launch_site_name}",
    icon=folium.Icon(color='blue', icon='info-sign')
).add_to(site_map)

# (Optional) Add a small label showing the distance right at the POI
folium.Marker(
    [poi_lat, poi_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html=f'<div style="font-size:12px; color:#d35400;"><b>{distance_km:.2f} KM</b></div>'
    )
).add_to(site_map)

# --- 5) Draw a line between the POI and the launch site ---
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [poi_lat, poi_lon]],
    weight=2,
    color="#d35400",
    opacity=0.85
).add_to(site_map)

# (Optional) emphasize the launch site
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup=f"{launch_site_name}<br>{launch_site_lat:.6f}, {launch_site_lon:.6f}",
    icon=folium.Icon(color='green', icon='rocket', prefix='fa')
).add_to(site_map)

site_map

After you plot distance lines to the proximities, you can answer the following questions easily:
- Are launch sites in close proximity to railways?
- Are launch sites in close proximity to highways?
- Are launch sites in close proximity to coastline?
- Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


### Are launch sites in close proximity to railways?

Mostly yes — but not all.

Kennedy Space Center (LC-39A) and Cape Canaveral (LC-40): connected to the NASA Railroad, which links to the U.S. national rail network for heavy rocket transport.

Vandenberg SLC-4E (California): near the Union Pacific coastal line, within a few kilometers.

Boca Chica (Texas): no direct rail connection — nearest rail is in Brownsville, about 25–30 km away.

- Conclusion:

Most launch sites are rail-accessible, except Boca Chica.

### Are launch sites in close proximity to highways?

 Yes — all of them.

All pads lie near major roads used for heavy-load transport and crew access.

LC-39A / LC-40 → Florida SR-3, SR-528

Vandenberg → California Highway 1

Boca Chica → Texas Highway 4

- Conclusion:

Every launch site has direct highway access (typically within 1–5 km).

### Are launch sites in close proximity to the coastline?

 Yes — deliberately so.

All sites are on or near the ocean (Atlantic, Pacific, or Gulf of Mexico).

Typical distance to open water: < 2–3 km.

Launching over the sea allows debris to fall safely away from populated areas.

- Conclusion:

All launch sites are coastal, a core safety requirement.

### Do launch sites keep a certain distance from cities?

 Yes — always.
 | Site                         | Nearest City                             | Distance    |
| :--------------------------- | :--------------------------------------- | :---------- |
| **Kennedy / Cape Canaveral** | Titusville (~15–20 km), Orlando (~60 km) | Safe buffer |
| **Vandenberg**               | Lompoc (~15 km)                          | Safe buffer |
| **Boca Chica**               | Brownsville (~30 km)                     | Safe buffer |


### Overall Findings
| Aspect        | Proximity                           | Reason                    |
| :------------ | :---------------------------------- | :------------------------ |
| **Railways**  | Usually close *(except Boca Chica)* | Heavy transport logistics |
| **Highways**  | Always close                        | Ground access & transport |
| **Coastline** | Always very close                   | Over-ocean launch safety  |
| **Cities**    | Always distant                      | Population risk buffer    |


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.
